<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_0702RNN_binary_addtion_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リカレントニューラルネットワークによる足し算のデモ

- author: 浅川伸一
- date: 2021_0702
- filename: 2021_0702RNN_binary_addtion_demo.ipynb

a + b = ? という足し算を計算する問題


In [ ]:
import numpy as np
import sys
import copy    # for deepcopy

In [ ]:
def sigmoid(x):
    """シグモイド関数"""
    return 1./(1.+np.exp(-x))

def d_sigmoid(x):
    """シグモイド関数の微分"""
    return x * (1.-x)

binary_dim = 8                       # 2 の 8 乗 = 256 までの自然数について計算する
int2bin = {}
largest_number = pow(2, binary_dim)  # 最大値
binary = np.unpackbits(              # 最大値までを 2 進数として表現しておく
    np.array([range(largest_number)], dtype=np.uint8).T, axis=1)

for i in range(largest_number):
    int2bin[i] = binary[i]

# ハイパーパラメータ
lr = 0.5     # 学習率 learning ratio
iter_max = 5 * 10 ** 3  # 最大繰り返し数
interval = iter_max >> 5
n_inp = 2    #入力層ニューロン数
n_hid = 8    #中間層ニューロン数
n_out = 1    #出力層ニューロン数

# 推定すべきパラメータ，結合係数の初期値を乱数で初期値
W_ih = 2 * np.random.random((n_inp, n_hid)) - 1.  # 入力層から中間層への結合係数
W_ho = 2 * np.random.random((n_hid, n_out)) - 1.  # 中間層から出力層への結合係数
W_hh = 2 * np.random.random((n_hid, n_hid)) - 1.  # 中間層から中間層への再帰結合

# 勾配 (微分) を保存しておくための領域
dW_ih = np.zeros_like(W_ih)
dW_ho = np.zeros_like(W_ho)
dW_hh = np.zeros_like(W_hh)

In [ ]:
for i in range(iter_max):
    # 乱数を発生させて a, b を定める
    a_int = np.random.randint(largest_number / 2)
    a = int2bin[a_int]
    b_int = np.random.randint(largest_number / 2)
    b = int2bin[b_int]
    c_int = a_int + b_int  # c 正解
    c = int2bin[c_int]
    d = np.zeros_like(c)

    total_err = 0
    O_deltas = list()
    H_list = list()
    H_list.append(np.zeros(n_hid))

    # 前向き処理
    for pos in range(binary_dim):
        k = binary_dim - pos - 1
        X = np.array([[a[k], b[k]]])     # X 入力層表現
        y = np.array([[c[k]]]).T         # y 出力層表現

        # H 中間層
        H = sigmoid(np.dot(X, W_ih) + np.dot(H_list[-1], W_hh))
        O = sigmoid(np.dot(H, W_ho))     # O 出力

        Delta = y - O                    # 誤差の計算 
        O_deltas.append((Delta) * d_sigmoid(O))
        total_err += np.mean(Delta[0] ** 2)
        
        d[k] = np.round(O[0][0])         # 予測出力のために d に予測値を保存
        H_list.append(copy.deepcopy(H))  # 次の時刻の処理のために中間層の値を保存

    # バックプロパーゲションによる学習
    future_H_delta = np.zeros(n_hid)
    for pos in range(binary_dim):
        X      = np.array([[a[pos], b[pos]]])
        H      = H_list[- pos - 1]
        H_prev = H_list[- pos - 2]

        O_delta = O_deltas[- pos - 1]
        H_delta = (future_H_delta.dot(W_hh.T) + O_delta.dot(W_ho.T)) * d_sigmoid(H)

        dW_ho += np.atleast_2d(     H).T.dot(O_delta)
        dW_hh += np.atleast_2d(H_prev).T.dot(H_delta)
        dW_ih += X.T.dot(H_delta)

        future_H_delta = H_delta

    W_ho += lr * dW_ho  # 結合係数の更新 中間層から出力層への結合係数
    W_hh += lr * dW_hh  # 中間層から中間層への結合係数
    W_ih += lr * dW_ih  # 入力層から中間層への結合係数

    dW_ho, dW_hh, dW_ih = 0, 0, 0  # 再初期化

    if i % interval == 0:
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2,index)
        print(f'反復回数:{i:>5d}, 誤差: {total_err:.3f}', end=' ')
        print(f'問題: {a_int} + {b_int} = {c_int}, 予測:{out}') 